In [5]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [19]:
#  declare a class of our neural Network
class NeuralNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.l1 = torch.nn.Linear(input_size, hidden_size)    #First Layer                           
        self.l2 = torch.nn.Linear(hidden_size, hidden_size)      #Second Layer Activation
        self.l3 = torch.nn.Linear(hidden_size, output_size)
        self.activation = torch.nn.ReLU() # in this case Relu would be fine! but may be for other one you should be more carefull
        self.init_weights()
        
        def init_weights(self):
            torch.nn.init.kaiming_normal_(self.l1.weight)
            torch.nn.init.kaiming_normal_(self.l2.weight)
        
        
        def forward(self, X):                          
            out = self.l1(X)
            out = self.activation(out)
            out = self.l2(out)
            out = self.activation(out)
            out = self.l3(out)
            return out
        

In [20]:
#  number of input neurons based on image pixels
input_size = 784
hidden_size = 392
output_size = 10
epochs = 25
batch_size = 100
learning_rate = 0.001

In [12]:
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)

In [13]:
test_dataset = datasets.MNIST(root='./data', train=False,transform=transforms.ToTensor())

In [21]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)
# our data would be iterable by loading it to a loader from torch.utils.data.DataLoader() 
# by means of shuffle = True, our data would be independent to the order of data